# 1- Check features after reducing cardinality
(the tables used in this notebook are created by `scripts/model1_preprocess_3`)

In [1]:
import pandas as pd
import numpy as np

In [2]:
techniques_df = pd.read_pickle ('../scripts/tmp_m1pp_technique.pkl')
groups_df = pd.read_pickle ('../scripts/tmp_m1pp_group.pkl')

techniques_org_df = pd.read_pickle ('../scripts/tmp_m1pp_technique_org.pkl')
groups_org_df = pd.read_pickle ('../scripts/tmp_m1pp_group_org.pkl')

In [25]:
techniques_df

,technique_ID,input_technique_data_sources,input_technique_detection_name,input_technique_mitigation_id,input_technique_platforms,input_technique_software_id,input_technique_tactics,input_technique_description,input_technique_interaction_rate
0,T1548,"[command_command_execution, file_file_metadata...","[command_execution, file_metadata, file_modifi...","[m1047, m1038, m1028, m1026, m1022, other]","[linux, windows, macos]",[],"[defense_evasion, privilege_escalation]","[-0.36630338, -0.24036503, -0.36260876, 0.0984...",-0.516771
1,T1548.002,"[command_command_execution, process_process_cr...","[command_execution, process_creation, windows_...","[m1047, m1026, other]",[windows],"[s0584, s0640, s0606, s0570, s1068, s0089, s10...","[defense_evasion, privilege_escalation]","[-0.5305501, -0.43114784, -0.67361057, 0.16222...",0.420987
2,T1548.004,"[process_os_api_execution, process_process_cre...","[os_api_execution, process_creation]",[m1038],[macos],[s0402],"[defense_evasion, privilege_escalation]","[-0.57463825, -0.65824693, -0.23797776, -0.254...",-0.516771
3,T1548.001,"[command_command_execution, file_file_metadata...","[command_execution, file_metadata, file_modifi...",[m1028],"[linux, macos]",[other],"[defense_evasion, privilege_escalation]","[-0.7420097, -0.461457, -0.45979652, -0.116038...",-0.516771
4,T1548.003,"[command_command_execution, file_file_modifica...","[command_execution, file_modification, process...","[m1028, m1026, m1022]","[linux, macos]","[s0154, s0279, other]","[defense_evasion, privilege_escalation]","[-0.8037999, -0.6815256, 0.18332177, 0.0635958...",-0.516771
...,...,...,...,...,...,...,...,...,...
602,T1102.002,"[network_traffic_network_connection_creation, ...","[network_connection_creation, network_traffic_...","[m1031, other]","[linux, windows, macos]","[s0128, s0657, s0660, s0126, s0244, s0046, s05...",[command_and_control],"[0.0067764707, -0.35248113, -0.62356573, 0.294...",1.202453
603,T1102.001,"[network_traffic_network_traffic_content, netw...","[network_traffic_content, network_traffic_flow]","[m1031, other]","[linux, windows, macos]","[s0373, s0128, s0674, s0531, s0455, s0013, s01...",[command_and_control],"[-0.7252811, -0.3269149, -0.49607247, 0.198911...",-0.204185
604,T1102.003,"[network_traffic_network_connection_creation, ...","[network_connection_creation, network_traffic_...","[m1031, other]","[linux, windows, macos]","[s0455, other]",[command_and_control],"[-0.11939225, -0.35763258, -0.46435547, 0.5083...",-0.516771
605,T1047,"[command_command_execution, network_traffic_ne...","[command_execution, network_connection_creatio...","[m1040, m1038, m1026, m1018]",[windows],"[s0331, s0373, s0640, s0534, s1070, s1068, s00...",[execution],"[-0.63444924, -0.15962608, -0.2825894, 0.07326...",2.296505


In [3]:
techniques_df.columns

Index(['technique_ID', 'input_technique_data_sources',
       'input_technique_detection_name', 'input_technique_mitigation_id',
       'input_technique_platforms', 'input_technique_software_id',
       'input_technique_tactics', 'input_technique_description',
       'input_technique_interaction_rate'],
      dtype='object')

In [4]:
techniques_org_df['input_technique_software_id'][0]

['']

In [5]:
from statistics import mode
def get_feature_length_stats(df: pd.DataFrame()):
    for col in list (df.columns)[1:]:
        if not isinstance(df[col].iloc[0], list): print ('{col}: scalar'.format (col = col))
        else:
                list_lengths = df[col].apply(lambda x: len(x))
                avg_len = list_lengths.mean()
                mode_len = mode(list_lengths)
                max_len = list_lengths.max()
                print ('{col} \tmean_len: {mean_len}\tmode_len: {mode_len}\tmax_len: {max_len}'.format(col = col, mean_len= avg_len, mode_len = mode_len, max_len = max_len))
        
def get_vocab_size (df: pd.DataFrame):
    for feature_name in df.columns[1:]:
        if feature_name in ['input_group_description', 'input_technique_description', 'input_group_interaction_rate', 'input_technique_interaction_rate']: continue
        vocab_size = len(df[feature_name].explode().unique())
        print ('{feature_name}: vocab size = {vocab_size}'.format (feature_name = feature_name, vocab_size = vocab_size))

In [6]:
print ("BEFORE")
get_feature_length_stats (techniques_org_df)
get_feature_length_stats (groups_org_df)
print ("AFTER")
get_feature_length_stats (techniques_df)
get_feature_length_stats (groups_df)

BEFORE
input_technique_data_sources 	mean_len: 3.154859967051071	mode_len: 3	max_len: 14
input_technique_detection_name 	mean_len: 3.154859967051071	mode_len: 3	max_len: 14
input_technique_mitigation_id 	mean_len: 2.1301482701812193	mode_len: 1	max_len: 11
input_technique_platforms 	mean_len: 2.510708401976936	mode_len: 1	max_len: 10
input_technique_software_id 	mean_len: 14.179571663920923	mode_len: 1	max_len: 334
input_technique_tactics 	mean_len: 1.2932454695222406	mode_len: 1	max_len: 4
input_technique_description: scalar
input_group_software_id 	mean_len: 6.1911764705882355	mode_len: 1	max_len: 46
input_group_description: scalar
AFTER
input_technique_data_sources 	mean_len: 2.7528830313014825	mode_len: 1	max_len: 9
input_technique_detection_name 	mean_len: 2.7528830313014825	mode_len: 1	max_len: 9
input_technique_mitigation_id 	mean_len: 1.9588138385502472	mode_len: 1	max_len: 8
input_technique_platforms 	mean_len: 2.510708401976936	mode_len: 1	max_len: 10
input_technique_software

In [7]:
print ("BEFORE")
get_vocab_size (groups_org_df)
get_vocab_size (techniques_org_df)
print ("AFTER")
get_vocab_size (groups_df)
get_vocab_size (techniques_df)

BEFORE
input_group_software_id: vocab size = 464
input_technique_data_sources: vocab size = 106
input_technique_detection_name: vocab size = 106
input_technique_mitigation_id: vocab size = 44
input_technique_platforms: vocab size = 11
input_technique_software_id: vocab size = 636
input_technique_tactics: vocab size = 14
AFTER
input_group_software_id: vocab size = 224
input_group_tactics: vocab size = 15
input_technique_data_sources: vocab size = 15
input_technique_detection_name: vocab size = 15
input_technique_mitigation_id: vocab size = 17
input_technique_platforms: vocab size = 11
input_technique_software_id: vocab size = 272
input_technique_tactics: vocab size = 14


In [24]:
feature_name = 'input_technique_mitigation_id'
print ('BEFORE')
print (techniques_org_df[feature_name].explode(feature_name).value_counts())
print ('AFTER')
print (techniques_df[feature_name].explode(feature_name).value_counts())

BEFORE
input_technique_mitigation_id
m1026    105
          93
m1018     89
m1056     82
m1047     78
m1038     66
m1042     59
m1022     56
m1031     55
m1040     46
m1027     44
m1017     41
m1032     41
m1037     40
m1028     39
m1030     35
m1051     32
m1041     29
m1054     24
m1021     23
m1024     20
m1045     20
m1035     16
m1049     15
m1015     14
m1048     12
m1029     11
m1046     11
m1050     11
m1053      9
m1057      9
m1013      9
m1043      8
m1036      7
m1025      7
m1052      7
m1033      6
m1016      5
m1020      4
m1034      4
m1019      4
m1044      3
m1055      2
m1039      2
Name: count, dtype: int64
AFTER
input_technique_mitigation_id
other    220
m1026    105
          93
m1018     89
m1056     82
m1047     78
m1038     66
m1042     59
m1022     56
m1031     55
m1040     46
m1027     44
m1017     41
m1032     41
m1037     40
m1028     39
m1030     35
Name: count, dtype: int64


---

In [9]:
for col in techniques_df.columns[1:-1]:
    print ('--------',col)
    feature_name = col
    print (techniques_df.explode(feature_name)[feature_name].value_counts())

-------- input_technique_data_sources
input_technique_data_sources
other                                                 312
command_command_execution                             268
process_process_creation                              226
network_traffic_network_traffic_content               108
file_file_modification                                104
process_os_api_execution                               94
file_file_creation                                     91
network_traffic_network_traffic_flow                   86
application_log_application_log_content                72
windows_registry_windows_registry_key_modification     65
network_traffic_network_connection_creation            61
module_module_load                                     54
file_file_access                                       47
file_file_metadata                                     44
                                                       39
Name: count, dtype: int64
-------- input_technique_detection_na

# 2 Check labels

In [10]:
labels_org_df = pd.read_pickle ('../scripts/tmp_m1_pp_label_org.pkl')
labels_df = pd.read_pickle ('../scripts/tmp_m1pp_label.pkl')

In [11]:
print (labels_org_df['label'].value_counts())
labels_org_df

label
0.0    49716
1.0     3052
Name: count, dtype: int64


,group_ID,technique_ID,label
0,G0099,T1548.002,0.0
1,G0099,T1134,0.0
2,G0099,T1134.002,0.0
3,G0099,T1134.001,0.0
4,G0099,T1531,0.0
...,...,...,...
52763,G0045,T1102.002,0.0
52764,G0045,T1102.001,0.0
52765,G0045,T1102.003,0.0
52766,G0045,T1047,1.0


In [12]:
print (labels_df['label'].value_counts())
labels_df

label
0.0    23113
1.0     2571
Name: count, dtype: int64


,group_ID,technique_ID,label
388,G0006,T1548.002,0.0
389,G0006,T1134,0.0
390,G0006,T1134.002,0.0
391,G0006,T1134.001,0.0
392,G0006,T1531,0.0
...,...,...,...
52763,G0045,T1102.002,0.0
52764,G0045,T1102.001,0.0
52765,G0045,T1102.003,0.0
52766,G0045,T1047,1.0


In [13]:
labels_org_df.columns == labels_df.columns

array([ True,  True,  True])

## 2b- Check resampled data

In [14]:
train_y_df = pd.read_csv ('../data/processed/model1/train_y_split.csv')
# train_cv_y_df = pd.read_csv ('../data/processed/model1/train_cv_y_split.csv')
cv_y_df = pd.read_csv ('../data/processed/model1/cv_y_split.csv')
# test_y_df = pd.read_csv ('../data/processed/model1/test_y_split.csv')
train_y_resampled_df = pd.read_csv ('../data/processed/model1/train_y_resampled.csv')


In [15]:
for df in [train_y_df, train_y_resampled_df,cv_y_df]:
    print (df.shape, 'no. of groups: {n}'.format(n =  df['group_ID'].nunique()))
    print (df['label'].value_counts())

(20480, 3) no. of groups: 54
label
0.0    18473
1.0     2007
Name: count, dtype: int64
(16007, 3) no. of groups: 54
label
0.0    12313
1.0     3694
Name: count, dtype: int64
(5204, 3) no. of groups: 14
label
0.0    4640
1.0     564
Name: count, dtype: int64


# 3 Check exported dataset

In [16]:
import tensorflow as tf

In [17]:
train_dataset = tf.data.Dataset.load ('../data/processed/model1/train_dataset/')
cv_dataset = tf.data.Dataset.load ('../data/processed/model1/cv_dataset/')
# test_dataset = tf.data.Dataset.load ('../data/processed/model1/test_dataset/')
inputs = cv_dataset.element_spec[0]

In [18]:
cv_dataset.element_spec[0]

{'input_technique_interaction_rate': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
 'input_technique_software_id': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_technique_platforms': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_group_tactics': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_technique_description': TensorSpec(shape=(768,), dtype=tf.float32, name=None),
 'input_technique_data_sources': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_group_interaction_rate': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
 'input_technique_tactics': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_group_software_id': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'input_group_description': TensorSpec(shape=(768,), dtype=tf.float32, name=None),
 'input_technique_mitigation_id': RaggedTensorSpec(TensorShape([None]), tf.string, 0, tf.int64),
 'in

In [19]:
input_technique = [item for item in list(inputs.keys()) if item.startswith ('input_technique')]
input_technique

['input_technique_interaction_rate',
 'input_technique_software_id',
 'input_technique_platforms',
 'input_technique_description',
 'input_technique_data_sources',
 'input_technique_tactics',
 'input_technique_mitigation_id',
 'input_technique_detection_name']

In [20]:
input_group = [item for item in list(inputs.keys()) if item.startswith ('input_group')]
input_group

['input_group_tactics',
 'input_group_interaction_rate',
 'input_group_software_id',
 'input_group_description']

In [21]:
# import tensorflow as tf
# count_zeros = 0
# count_ones = 0
# for features, labels in train_dataset:
#     # 'labels' will be the second element in each tuple
#     labels_array = labels.numpy()  # Convert labels tensor to numpy array
    
#     # Count occurrences of 0 and 1 in the labels
#     count_zeros += (labels_array == 0).sum()
#     count_ones += (labels_array == 1).sum()

# # Print the counts
# print(f"Number of zeros: {count_zeros}")
# print(f"Number of ones: {count_ones}")

In [22]:
# count_zeros = 0
# count_ones = 0
# for features, labels in cv_dataset:
#     # 'labels' will be the second element in each tuple
#     labels_array = labels.numpy()  # Convert labels tensor to numpy array
    
#     # Count occurrences of 0 and 1 in the labels
#     count_zeros += (labels_array == 0).sum()
#     count_ones += (labels_array == 1).sum()

# # Print the counts
# print(f"Number of zeros: {count_zeros}")
# print(f"Number of ones: {count_ones}")